In [ ]:
!curl --request GET --url host.docker.internal:21434/api/tags

In [ ]:
from langchain_ollama import ChatOllama

### llm 
local_llm="deepseek-r1:1.5b-qwen-distill-fp16"
llm = ChatOllama(model=local_llm, temperature=0, base_url="http://host.docker.internal:21434")
llm_json = ChatOllama(model=local_llm, temperature=0, return_json=True, base_url="http://host.docker.internal:21434")

In [ ]:
msg = llm.invoke("What is the capital of France?")
msg.content

__________________

# GROQ test

In [ ]:
import os

from groq import Groq
from dotenv import load_dotenv
load_dotenv("../../groq.env")


In [ ]:
client = Groq(
    # This is the default and can be omitted
    api_key=os.getenv("GROQ_API_KEY"),
)

In [ ]:
model_name = os.getenv("GROQ_LLM")
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "you are a helpful assistant."
        },
        {
            "role": "user",
            "content": "Explain the importance of fast language models",
        }
    ],
    model=model_name,
    reasoning_format={"type": "groq"},
    temperature=0.6,
    max_completion_tokens=131072,
    top_p=0.95,
    stream=True,
    stop=None,
    
)

In [ ]:

for chunk in chat_completion:
    print(chunk.choices[0].delta.content or "", end="")

__________


# GROQ langchain test

In [ ]:
import os
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, SystemMessage
from ollama_deep_researcher.prompts import (
    query_writer_instructions,
    summarizer_instructions,
    reflection_instructions,
    get_current_date,
)
from dotenv import load_dotenv
load_dotenv("../../groq.env")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
model_name = os.getenv("GROQ_LLM")

In [ ]:
current_date = get_current_date()
formatted_prompt = query_writer_instructions.format(
        current_date=current_date, research_topic="fast language models"
    )

In [ ]:
llm = ChatGroq(
    model=model_name,
    temperature=0,
    response_format={"type": "json_object"},
)
#llm_json_mode = llm.with_structured_output(method="json_mode")

In [ ]:
result = llm_json_mode.invoke(
    [
        SystemMessage(content=formatted_prompt),
        HumanMessage(content=f"Generate a query for web search:"),
    ]
)

In [ ]:
result2 = llm.invoke(
    [
        SystemMessage(content=formatted_prompt),
        HumanMessage(content=f"Generate a query for web search:"),
    ]
)

In [ ]:
print(result2.content)

_____

# tavily test (Used for making the crawler search)

In [ ]:
import os
from ollama_deep_researcher.utils import (
    deduplicate_and_format_sources,
    tavily_search,
    format_sources,
)
from dotenv import load_dotenv
load_dotenv("../../.env")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")

In [ ]:
search_results = tavily_search("what is the capital of France?", fetch_full_page=True, max_results=3)

In [ ]:
search_results

In [ ]:
search_str = deduplicate_and_format_sources(
            search_results,
            max_tokens_per_source=10000000,
            fetch_full_page=True,
        )

In [ ]:
print(search_str)

_______

# Test rets_info_scraper

In [1]:
from ollama_deep_researcher.retsinfo_crawl import retsinfo_search_and_crawl

In [2]:
if __name__ == "__main__":
    # Load environment variables from .env file
   result = await retsinfo_search_and_crawl(query="Justitsministerens bemyndigelse og retsgrund for indsamlinger", max_results=3)

    

[INIT].... → Crawl4AI 0.6.3 

[INIT].... → Crawl4AI 0.6.3 

[INIT].... → Crawl4AI 0.6.3 

[INIT].... → Crawl4AI 0.6.3 

[INIT].... → Crawl4AI 0.6.3 

...

[FETCH]... ↓ https://www.retsinformation.dk/eli/retsinfo/2016/10488                                               |
✓ | ⏱: 7.99s 

[SCRAPE].. ◆ https://www.retsinformation.dk/eli/retsinfo/2016/10488                                               |
✓ | ⏱: 0.02s 

[COMPLETE] ● https://www.retsinformation.dk/eli/retsinfo/2016/10488                                               |
✓ | ⏱: 8.01s 

Failed: https://www.retsinformation.dk/eli/lta/2006/555 -


UnboundLocalError: cannot access local variable 'id' where it is not associated with a value